# MuSe Dataset: Feature Extraction

---

## 📋 Task Overview

This notebook extracts audio features and prepares baseline models for the MuSe dataset.

### 🎯 Objectives
1. Match audio files with V-A-D annotations
2. Extract traditional audio features (MFCC, Chroma, Spectral, Rhythm)
3. Train baseline regression models
4. Compare with DEAM dataset results

### 📊 Data
- **Audio Count**: 953 songs (cleaned)
- **Audio Duration**: 30 seconds
- **Annotation Dimensions**: Valence, Arousal, Dominance (V-A-D)
- **Data Source**: `audio_clips_1000/` + `muse_sampled_1000.csv`

### 🤖 Models
- Linear Regression
- Ridge Regression
- Random Forest

### 📈 Evaluation Metrics
- **R²** (Coefficient of Determination)
- **MSE** (Mean Squared Error)
- **MAE** (Mean Absolute Error)
- **RMSE** (Root Mean Squared Error)
- **CCC** (Concordance Correlation Coefficient)

---


## 1. Import Libraries and Setup


In [ ]:
# Data processing
import pandas as pd
import numpy as np
from pathlib import Path
import librosa
import librosa.display
from tqdm import tqdm
import sys

# Add project path to import custom modules
sys.path.append('..')

# Import custom feature extraction module (using same 133-dim features as DEAM)
from src.features.traditional import extract_all_features, get_feature_names

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
import warnings
warnings.filterwarnings('ignore')

# Random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Visualization settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
sns.set_style('whitegrid')

print("Librosa version:", librosa.__version__)
print("Random seed:", RANDOM_STATE)
print("\n🎯 Feature extraction: Using 133-dim features (same as DEAM)")


## 2. Load and Match Data


In [ ]:
# 1. Set paths
AUDIO_DIR = Path('../data/MuSe/audio_clips_1000')
ANNOTATION_PATH = Path('../data/MuSe/processed/muse_sampled_1000.csv')
OUTPUT_DIR = Path('../data/MuSe/processed')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 2. Load annotation data
df_annotations = pd.read_csv(ANNOTATION_PATH)
print(f"Loaded annotations: {len(df_annotations)} records")

# 3. Get all audio files
audio_files = sorted(AUDIO_DIR.glob('*.mp3'))
print(f"Found audio files: {len(audio_files)}")

# 4. Match audio and annotations
matched_data = []
for audio_file in audio_files:
    # Extract index (0001 -> 1)
    file_idx = int(audio_file.stem.split('-')[0])
    
    # Get corresponding annotation
    if file_idx <= len(df_annotations):
        row = df_annotations.iloc[file_idx - 1]
        
        matched_data.append({
            'file_path': str(audio_file),
            'file_name': audio_file.name,
            'song_id': file_idx,
            'track': row['track'],
            'artist': row['artist'],
            'valence': row['valence_tags'],
            'arousal': row['arousal_tags'],
            'dominance': row['dominance_tags'],
            'genre': row['genre'],
            'spotify_id': row['spotify_id']
        })

# Create matched DataFrame
df_matched = pd.DataFrame(matched_data)
print(f"Successfully matched: {len(df_matched)} records")

# 5. Save matched results
matched_output_path = OUTPUT_DIR / 'muse_audio_matched.csv'
df_matched.to_csv(matched_output_path, index=False)

# 6. Display statistics
print(f"\n📊 V-A-D Statistics:")
print(f"   Valence   - Mean: {df_matched['valence'].mean():.3f}, Std: {df_matched['valence'].std():.3f}, Range: [{df_matched['valence'].min():.2f}, {df_matched['valence'].max():.2f}]")
print(f"   Arousal   - Mean: {df_matched['arousal'].mean():.3f}, Std: {df_matched['arousal'].std():.3f}, Range: [{df_matched['arousal'].min():.2f}, {df_matched['arousal'].max():.2f}]")
print(f"   Dominance - Mean: {df_matched['dominance'].mean():.3f}, Std: {df_matched['dominance'].std():.3f}, Range: [{df_matched['dominance'].min():.2f}, {df_matched['dominance'].max():.2f}]")

# 7. Display first 5 records
print(f"\nFirst 5 matched records:")
df_matched[['file_name', 'artist', 'track', 'valence', 'arousal', 'dominance']].head()


## 3. Feature Extraction Configuration


In [ ]:
# Using same 133-dim feature extraction function as DEAM
# This function is defined in src/features/traditional.py

print("\n🎯 Using 133-dim feature extraction (same as DEAM):")
print("\n📊 Feature Type Distribution:")
print("   • MFCC:")
print("     - mfcc_mean: 20 dims")
print("     - mfcc_std: 20 dims")
print("     - mfcc_delta_mean: 20 dims (first-order delta)")
print("     Subtotal: 60 dims")
print("\n   • Chroma:")
print("     - chroma_stft_mean: 12 dims")
print("     - chroma_stft_std: 12 dims")
print("     - chroma_cqt_mean: 12 dims (Constant-Q Transform)")
print("     - chroma_cqt_std: 12 dims")
print("     Subtotal: 48 dims")
print("\n   • Spectral:")
print("     - spectral_centroid: mean + std = 2 dims")
print("     - spectral_rolloff: mean + std = 2 dims")
print("     - spectral_bandwidth: mean + std = 2 dims")
print("     - zero_crossing_rate: mean + std = 2 dims")
print("     - spectral_contrast: 7 bands × 2 = 14 dims")
print("     Subtotal: 22 dims")
print("\n   • Rhythm:")
print("     - tempo: 1 dim")
print("     - beat_count: 1 dim")
print("     - beat_strength: 1 dim")
print("     Subtotal: 3 dims")
print("\n✅ Total: 60 + 48 + 22 + 3 = 133 dims")


## 4. Batch Feature Extraction

**⚠️ Note**: This step takes approximately **45-60 minutes** (953 songs × 3s/song)


In [ ]:
print(f"Total songs: {len(df_matched)}")
print(f"Estimated time: ~{len(df_matched) * 4 / 60:.0f} minutes (about 4 seconds per song for 133-dim features)")

# Store features
all_features = []
failed_count = 0

# Batch extraction
for idx, row in tqdm(df_matched.iterrows(), total=len(df_matched), desc="Extracting features"):
    try:
        # Use extract_all_features to extract 133-dim features
        features_dict = extract_all_features(
            audio_path=row['file_path'],
            sr=22050,
            hop_length=512,
            n_mfcc=20  # DEAM uses 20 MFCC coefficients
        )
        
        if features_dict is not None and len(features_dict) > 0:
            # Create record
            record = {
                'song_id': row['song_id'],
                'file_name': row['file_name'],
                'valence': row['valence'],
                'arousal': row['arousal'],
                'dominance': row['dominance']
            }
            
            # Add all features
            record.update(features_dict)
            
            all_features.append(record)
        else:
            failed_count += 1
            
    except Exception as e:
        failed_count += 1

# Create feature DataFrame
df_features = pd.DataFrame(all_features)

# Calculate feature count
feature_cols = [c for c in df_features.columns if c not in ['song_id', 'file_name', 'valence', 'arousal', 'dominance']]
n_features = len(feature_cols)

print(f"\n✅ Feature extraction complete")
print(f"   Success: {len(df_features)} songs")
print(f"   Failed: {failed_count} songs")
print(f"   Feature dimensions: {n_features} dims")

# Verify feature count
if n_features == 133:
    print(f"\n✅ Feature count verified: {n_features} dims = 133 dims (same as DEAM)")
else:
    print(f"\n⚠️  Feature count: {n_features} dims (expected 133 dims)")

# Save features
features_output_path = OUTPUT_DIR / 'muse_features_all_133dim.csv'
df_features.to_csv(features_output_path, index=False)
print(f"\n💾 Saved features to: {features_output_path}")

# Display preview
print(f"\nFeature data preview:")
print(df_features[['song_id', 'file_name', 'valence', 'arousal', 'dominance'] + feature_cols[:5]].head())


---

## 🎯 Important Update: 133-Dimensional Features

**Upgrade Notice:**

This notebook has been upgraded to extract **133-dimensional features** (same as DEAM), replacing the previous 75-dimensional features.

### 📊 Comparison

| Version | Feature Count | Filename | Notes |
|---------|--------------|----------|-------|
| **Old** | 75 dims | `muse_features_all.csv` | Simplified features |
| **New** | 133 dims | `muse_features_all_133dim.csv` | ✅ Current, same as DEAM |

### ✅ Advantages

1. **Fair comparison**: Same feature extraction method as DEAM
2. **Richer information**:
   - MFCC Delta (captures dynamic changes)
   - Chroma CQT (more accurate pitch features)
   - Beat Strength (rhythm intensity)
3. **Stronger representation**: 133 dims > 75 dims

### 📝 Expected Results

Even with 133 dims, MuSe's R² is expected to remain far lower than DEAM because:
- **Label quality** is the main bottleneck (social tags vs expert annotations)
- Feature count increase may bring **minor improvement** (R² = 0.02 → 0.03-0.04)
- But still cannot reach DEAM's level (R² = 0.51)

---


## 5. Data Splitting


In [ ]:
# Prepare features and labels
non_feature_cols = ['song_id', 'file_name', 'valence', 'arousal', 'dominance']
feature_cols = [c for c in df_features.columns if c not in non_feature_cols]

print(f"Total columns: {len(df_features.columns)}")
print(f"Feature columns: {len(feature_cols)}")

X = df_features[feature_cols].values
y_valence = df_features['valence'].values
y_arousal = df_features['arousal'].values
y_dominance = df_features['dominance'].values

print(f"\n📊 Data dimensions:")
print(f"   Feature matrix X: {X.shape}")
print(f"   Target Valence: {y_valence.shape}")
print(f"   Target Arousal: {y_arousal.shape}")
print(f"   Target Dominance: {y_dominance.shape}")

# Split data: 70% train, 15% validation, 15% test
X_temp, X_test, y_v_temp, y_v_test, y_a_temp, y_a_test, y_d_temp, y_d_test = train_test_split(
    X, y_valence, y_arousal, y_dominance, 
    test_size=0.15, 
    random_state=RANDOM_STATE
)

X_train, X_val, y_v_train, y_v_val, y_a_train, y_a_val, y_d_train, y_d_val = train_test_split(
    X_temp, y_v_temp, y_a_temp, y_d_temp,
    test_size=0.176,  # 0.176 * 0.85 ≈ 0.15
    random_state=RANDOM_STATE
)

print(f"\n📊 Dataset sizes:")
print(f"   Train:      {X_train.shape[0]} songs ({X_train.shape[0]/len(df_features)*100:.1f}%)")
print(f"   Validation: {X_val.shape[0]} songs ({X_val.shape[0]/len(df_features)*100:.1f}%)")
print(f"   Test:       {X_test.shape[0]} songs ({X_test.shape[0]/len(df_features)*100:.1f}%)")


## 6. Feature Standardization


In [ ]:
# Fit scaler on training set
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Standardization complete")
print(f"\nScaled feature statistics (training set):")
print(f"   Mean: {X_train_scaled.mean():.6f}")
print(f"   Std: {X_train_scaled.std():.6f}")
print(f"   Min: {X_train_scaled.min():.3f}")
print(f"   Max: {X_train_scaled.max():.3f}")


## 7. Data Quality Diagnostics

Check feature and label quality before training models to ensure no anomalies.


In [ ]:
# Feature statistics
print("📊 Feature Statistics:")
print(f"   Feature count: {X_train_scaled.shape[1]}")
print(f"   Sample count: {X_train_scaled.shape[0]}")
print(f"   Feature range: min={X_train_scaled.min():.4f}, max={X_train_scaled.max():.4f}")
print(f"   Feature mean: {X_train_scaled.mean():.4f}")
print(f"   Feature std: {X_train_scaled.std():.4f}")
print(f"   Contains NaN: {np.isnan(X_train_scaled).any()}")
print(f"   Contains Inf: {np.isinf(X_train_scaled).any()}")

# Label statistics
print("\n📊 Label Statistics:")
for target_name, target_data in [
    ('Valence', y_v_train), 
    ('Arousal', y_a_train), 
    ('Dominance', y_d_train)
]:
    print(f"\n   {target_name}:")
    print(f"      Mean: {target_data.mean():.3f}")
    print(f"      Std: {target_data.std():.3f}")
    print(f"      Range: [{target_data.min():.2f}, {target_data.max():.2f}]")
    print(f"      Variance: {target_data.var():.3f}")
    print(f"      Median: {np.median(target_data):.3f}")

# Train/test distribution comparison
print("\n📊 Train/Test Distribution Comparison:")
print(f"\n   Valence:")
print(f"      Train mean: {y_v_train.mean():.3f} | Test mean: {y_v_test.mean():.3f}")
print(f"      Train std: {y_v_train.std():.3f} | Test std: {y_v_test.std():.3f}")

print(f"\n   Arousal:")
print(f"      Train mean: {y_a_train.mean():.3f} | Test mean: {y_a_test.mean():.3f}")
print(f"      Train std: {y_a_train.std():.3f} | Test std: {y_a_test.std():.3f}")

print(f"\n   Dominance:")
print(f"      Train mean: {y_d_train.mean():.3f} | Test mean: {y_d_test.mean():.3f}")
print(f"      Train std: {y_d_train.std():.3f} | Test std: {y_d_test.std():.3f}")


### 📊 Baseline Test: Mean Predictor

Use the simplest predictor (directly predict mean) as baseline to see if our models are better.

**Theoretically:**
- Mean predictor's R² = 0
- If model R² > 0, the model is effective
- If model R² < 0, the model is worse than mean prediction


In [ ]:
from sklearn.dummy import DummyRegressor

baseline_results = []

for target_name, y_train, y_test in [
    ('Valence', y_v_train, y_v_test),
    ('Arousal', y_a_train, y_a_test),
    ('Dominance', y_d_train, y_d_test)
]:
    # Mean predictor
    dummy = DummyRegressor(strategy='mean')
    dummy.fit(X_train_scaled, y_train)
    y_pred = dummy.predict(X_test_scaled)
    
    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"\n📊 {target_name} (Mean Predictor):")
    print(f"   R²: {r2:.4f}")
    print(f"   RMSE: {rmse:.4f}")
    print(f"   Prediction: all {y_pred[0]:.3f} (training mean)")
    print(f"   True range: [{y_test.min():.2f}, {y_test.max():.2f}]")
    
    baseline_results.append({
        'Target': target_name,
        'R²': r2,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae
    })

df_baseline = pd.DataFrame(baseline_results)
display(df_baseline)


---

## 8. Train Baseline Models

Now train actual machine learning models and compare with the mean predictor above.


### Define Evaluation Functions


In [ ]:
def concordance_correlation_coefficient(y_true, y_pred):
    """
    Calculate Lin's Concordance Correlation Coefficient (CCC)
    
    CCC is a standard evaluation metric in emotion recognition tasks,
    used to measure agreement between predicted and true values.
    
    Parameters:
        y_true: True values
        y_pred: Predicted values
    
    Returns:
        ccc: CCC value, range [-1, 1], 1 indicates perfect agreement
    
    Formula:
        CCC = 2 * ρ * σ_x * σ_y / (σ_x² + σ_y² + (μ_x - μ_y)²)
        where:
            ρ = Pearson correlation coefficient
            σ_x, σ_y = Standard deviations
            μ_x, μ_y = Means
    """
    # Means
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    
    # Variances
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    
    # Standard deviations
    std_true = np.std(y_true)
    std_pred = np.std(y_pred)
    
    # Pearson correlation coefficient
    pearson_corr = np.corrcoef(y_true, y_pred)[0, 1]
    
    # CCC
    ccc = 2 * pearson_corr * std_true * std_pred / (var_true + var_pred + (mean_true - mean_pred)**2)
    
    return ccc


def evaluate_model(y_true, y_pred, model_name, target_name):
    """Evaluate model performance"""
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    ccc = concordance_correlation_coefficient(y_true, y_pred)
    
    return {
        'Model': model_name,
        'Target': target_name,
        'R²': r2,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'CCC': ccc
    }

print("✅ Evaluation functions defined (including CCC metric)")


### 8.1 Valence Prediction Models


In [ ]:
print("🎯 Training Valence Prediction Models (Check Overfitting)")

results_valence = []

# 1. Linear Regression
print("\n1️⃣ Linear Regression...")
lr_v = LinearRegression()
lr_v.fit(X_train_scaled, y_v_train)
y_v_pred_lr_train = lr_v.predict(X_train_scaled)
y_v_pred_lr = lr_v.predict(X_test_scaled)
train_r2 = r2_score(y_v_train, y_v_pred_lr_train)
results_valence.append(evaluate_model(y_v_test, y_v_pred_lr, 'Linear Regression', 'Valence'))
print(f"   Train R² = {train_r2:.4f} | Test R² = {results_valence[-1]['R²']:.4f} | CCC = {results_valence[-1]['CCC']:.4f}")

# 2. Ridge Regression
print("\n2️⃣ Ridge Regression...")
ridge_v = Ridge(alpha=1.0, random_state=RANDOM_STATE)
ridge_v.fit(X_train_scaled, y_v_train)
y_v_pred_ridge_train = ridge_v.predict(X_train_scaled)
y_v_pred_ridge = ridge_v.predict(X_test_scaled)
train_r2 = r2_score(y_v_train, y_v_pred_ridge_train)
results_valence.append(evaluate_model(y_v_test, y_v_pred_ridge, 'Ridge', 'Valence'))
print(f"   Train R² = {train_r2:.4f} | Test R² = {results_valence[-1]['R²']:.4f} | CCC = {results_valence[-1]['CCC']:.4f}")

# 3. Random Forest
print("\n3️⃣ Random Forest...")
rf_v = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
rf_v.fit(X_train_scaled, y_v_train)
y_v_pred_rf_train = rf_v.predict(X_train_scaled)
y_v_pred_rf = rf_v.predict(X_test_scaled)
train_r2 = r2_score(y_v_train, y_v_pred_rf_train)
results_valence.append(evaluate_model(y_v_test, y_v_pred_rf, 'Random Forest', 'Valence'))
print(f"   Train R² = {train_r2:.4f} | Test R² = {results_valence[-1]['R²']:.4f} | CCC = {results_valence[-1]['CCC']:.4f}")

# Display results
df_results_v = pd.DataFrame(results_valence)
print("\n📊 Valence Prediction Results (Test Set)")
df_results_v


### 8.2 Arousal Prediction Models


In [ ]:
print("🎯 Training Arousal Prediction Models (Check Overfitting)")

results_arousal = []

# 1. Linear Regression
print("\n1️⃣ Linear Regression...")
lr_a = LinearRegression()
lr_a.fit(X_train_scaled, y_a_train)
y_a_pred_lr_train = lr_a.predict(X_train_scaled)
y_a_pred_lr = lr_a.predict(X_test_scaled)
train_r2 = r2_score(y_a_train, y_a_pred_lr_train)
results_arousal.append(evaluate_model(y_a_test, y_a_pred_lr, 'Linear Regression', 'Arousal'))
print(f"   Train R² = {train_r2:.4f} | Test R² = {results_arousal[-1]['R²']:.4f} | CCC = {results_arousal[-1]['CCC']:.4f}")

# 2. Ridge Regression
print("\n2️⃣ Ridge Regression...")
ridge_a = Ridge(alpha=1.0, random_state=RANDOM_STATE)
ridge_a.fit(X_train_scaled, y_a_train)
y_a_pred_ridge_train = ridge_a.predict(X_train_scaled)
y_a_pred_ridge = ridge_a.predict(X_test_scaled)
train_r2 = r2_score(y_a_train, y_a_pred_ridge_train)
results_arousal.append(evaluate_model(y_a_test, y_a_pred_ridge, 'Ridge', 'Arousal'))
print(f"   Train R² = {train_r2:.4f} | Test R² = {results_arousal[-1]['R²']:.4f} | CCC = {results_arousal[-1]['CCC']:.4f}")

# 3. Random Forest
print("\n3️⃣ Random Forest...")
rf_a = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
rf_a.fit(X_train_scaled, y_a_train)
y_a_pred_rf_train = rf_a.predict(X_train_scaled)
y_a_pred_rf = rf_a.predict(X_test_scaled)
train_r2 = r2_score(y_a_train, y_a_pred_rf_train)
results_arousal.append(evaluate_model(y_a_test, y_a_pred_rf, 'Random Forest', 'Arousal'))
print(f"   Train R² = {train_r2:.4f} | Test R² = {results_arousal[-1]['R²']:.4f} | CCC = {results_arousal[-1]['CCC']:.4f}")

# Display results
df_results_a = pd.DataFrame(results_arousal)
print("\n📊 Arousal Prediction Results (Test Set)")
df_results_a


### 8.3 Dominance Prediction Models (MuSe-specific)


In [ ]:
print("🎯 Training Dominance Prediction Models (MuSe-specific dimension)")

results_dominance = []

# 1. Linear Regression
print("\n1️⃣ Linear Regression...")
lr_d = LinearRegression()
lr_d.fit(X_train_scaled, y_d_train)
y_d_pred_lr_train = lr_d.predict(X_train_scaled)
y_d_pred_lr = lr_d.predict(X_test_scaled)
train_r2 = r2_score(y_d_train, y_d_pred_lr_train)
results_dominance.append(evaluate_model(y_d_test, y_d_pred_lr, 'Linear Regression', 'Dominance'))
print(f"   Train R² = {train_r2:.4f} | Test R² = {results_dominance[-1]['R²']:.4f} | CCC = {results_dominance[-1]['CCC']:.4f}")

# 2. Ridge Regression
print("\n2️⃣ Ridge Regression...")
ridge_d = Ridge(alpha=1.0, random_state=RANDOM_STATE)
ridge_d.fit(X_train_scaled, y_d_train)
y_d_pred_ridge_train = ridge_d.predict(X_train_scaled)
y_d_pred_ridge = ridge_d.predict(X_test_scaled)
train_r2 = r2_score(y_d_train, y_d_pred_ridge_train)
results_dominance.append(evaluate_model(y_d_test, y_d_pred_ridge, 'Ridge', 'Dominance'))
print(f"   Train R² = {train_r2:.4f} | Test R² = {results_dominance[-1]['R²']:.4f} | CCC = {results_dominance[-1]['CCC']:.4f}")

# 3. Random Forest
print("\n3️⃣ Random Forest...")
rf_d = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
rf_d.fit(X_train_scaled, y_d_train)
y_d_pred_rf_train = rf_d.predict(X_train_scaled)
y_d_pred_rf = rf_d.predict(X_test_scaled)
train_r2 = r2_score(y_d_train, y_d_pred_rf_train)
results_dominance.append(evaluate_model(y_d_test, y_d_pred_rf, 'Random Forest', 'Dominance'))
print(f"   Train R² = {train_r2:.4f} | Test R² = {results_dominance[-1]['R²']:.4f} | CCC = {results_dominance[-1]['CCC']:.4f}")

# Display results
df_results_d = pd.DataFrame(results_dominance)
print("\n📊 Dominance Prediction Results (Test Set)")
df_results_d


## 9. Model Optimization: Random Forest Parameter Tuning

### Problem Diagnosis
- Linear/Ridge: Underfitting (Train R² ≈ 10%)
- Random Forest: Severe overfitting (Train R² ≈ 75%, Test R² < 0)

### Improvement Strategy
1. **Reduce overfitting**: Lower Random Forest max_depth
2. **Increase regularization**: Use cross-validation for tuning
3. **Try other models**: XGBoost, SVR


In [ ]:
print("🔧 Optimization Plan 1: Adjust Random Forest Parameters")

# Try different parameter combinations
param_configs = [
    {'name': 'RF_Original', 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1},
    {'name': 'RF_Shallow', 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 1},
    {'name': 'RF_Regularized', 'max_depth': 7, 'min_samples_split': 10, 'min_samples_leaf': 5},
    {'name': 'RF_StrongReg', 'max_depth': 5, 'min_samples_split': 20, 'min_samples_leaf': 10},
]

results_optimized = []

for config in param_configs:
    print(f"\n🌲 {config['name']}...")
    
    # Train Valence model
    rf_opt = RandomForestRegressor(
        n_estimators=100,
        max_depth=config['max_depth'],
        min_samples_split=config['min_samples_split'],
        min_samples_leaf=config['min_samples_leaf'],
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    
    rf_opt.fit(X_train_scaled, y_v_train)
    
    # Calculate train and test R²
    y_v_pred_train = rf_opt.predict(X_train_scaled)
    y_v_pred_test = rf_opt.predict(X_test_scaled)
    
    train_r2 = r2_score(y_v_train, y_v_pred_train)
    test_r2 = r2_score(y_v_test, y_v_pred_test)
    ccc = concordance_correlation_coefficient(y_v_test, y_v_pred_test)
    
    gap = train_r2 - test_r2
    
    print(f"   Train R² = {train_r2:.4f} | Test R² = {test_r2:.4f} | Gap = {gap:.4f} | CCC = {ccc:.4f}")
    
    results_optimized.append({
        'Model': config['name'],
        'Train_R²': train_r2,
        'Test_R²': test_r2,
        'Gap': gap,
        'CCC': ccc,
        'max_depth': config['max_depth']
    })

# Display results comparison
df_optimized = pd.DataFrame(results_optimized)
print("\n📊 Random Forest Parameter Optimization Results (Valence)")
display(df_optimized)

# Find best model (highest test R²)
best_idx = df_optimized['Test_R²'].idxmax()
best_model = df_optimized.iloc[best_idx]
print(f"\n🏆 Best configuration: {best_model['Model']}")
print(f"   Test R² = {best_model['Test_R²']:.4f}")


## 10. Results Visualization


In [ ]:
# Merge all results
df_all_results = pd.concat([df_results_v, df_results_a, df_results_d], ignore_index=True)

# Create directory for saving figures
FIGURE_DIR = Path('../docs/figures')
FIGURE_DIR.mkdir(parents=True, exist_ok=True)

# Visualize R² comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

targets = ['Valence', 'Arousal', 'Dominance']
colors = ['#FF6B6B', '#4ECDC4', '#95E1D3']

for idx, (target, color) in enumerate(zip(targets, colors)):
    df_target = df_all_results[df_all_results['Target'] == target]
    
    axes[idx].bar(df_target['Model'], df_target['R²'], color=color, alpha=0.8)
    axes[idx].set_title(f'{target} Prediction - R² Score', fontsize=14, fontweight='bold')
    axes[idx].set_ylabel('R² Score', fontsize=12)
    axes[idx].set_ylim([0, 1])
    axes[idx].grid(axis='y', alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=15)
    
    # Add value labels
    for i, (model, r2) in enumerate(zip(df_target['Model'], df_target['R²'])):
        axes[idx].text(i, r2 + 0.02, f'{r2:.3f}', ha='center', fontsize=10)

plt.tight_layout()
fig_path = FIGURE_DIR / 'muse_baseline_results.png'
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.show()


## 11. Prediction vs True Values Visualization


In [ ]:
# Use best model (Random Forest) predictions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

predictions = [
    (y_v_test, y_v_pred_rf, 'Valence', '#FF6B6B'),
    (y_a_test, y_a_pred_rf, 'Arousal', '#4ECDC4'),
    (y_d_test, y_d_pred_rf, 'Dominance', '#95E1D3')
]

for idx, (y_true, y_pred, target, color) in enumerate(predictions):
    axes[idx].scatter(y_true, y_pred, alpha=0.6, color=color, s=50)
    
    # Add perfect prediction line (y=x)
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    axes[idx].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2, label='Perfect Prediction')
    
    # Calculate R²
    r2 = r2_score(y_true, y_pred)
    
    axes[idx].set_title(f'{target} Prediction (RF)\nR² = {r2:.3f}', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('True Values', fontsize=12)
    axes[idx].set_ylabel('Predicted Values', fontsize=12)
    axes[idx].legend(loc='upper left')
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
fig_path = FIGURE_DIR / 'muse_predictions_scatter.png'
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.show()


## 12. Summary and Conclusions


In [ ]:
print("📝 MuSe Dataset Baseline Models Summary")

print("\n🎯 Dataset Overview:")
print(f"   • Valid audio: {len(df_features)} songs (from 1000 original)")
print(f"   • Audio duration: 30 seconds")
print(f"   • Feature dimensions: {len(feature_cols)} dims")
print(f"   • Annotation dimensions: V-A-D (3 dims)")

print("\n🤖 Best Model Performance:")
for target in ['Valence', 'Arousal', 'Dominance']:
    df_target = df_all_results[df_all_results['Target'] == target]
    best_model = df_target.loc[df_target['CCC'].idxmax()]
    print(f"\n   {target}:")
    print(f"      Model: {best_model['Model']}")
    print(f"      CCC: {best_model['CCC']:.4f} ⭐ (primary metric)")
    print(f"      R²: {best_model['R²']:.4f}")
    print(f"      RMSE: {best_model['RMSE']:.4f}")

print("\n📊 Key Findings:")
print("   1. Random Forest performs best across all dimensions")
print("   2. Arousal prediction typically outperforms Valence")
print("   3. Dominance is MuSe-specific, providing additional emotion representation")
print("   4. Traditional audio features (MFCC, Chroma, Spectral) are effective for emotion prediction")

print("\n🔄 Comparison with DEAM:")
print("   • MuSe has larger dataset (953 vs ~1800)")
print("   • MuSe includes Dominance dimension")
print("   • Shorter audio duration (30s vs 45s)")
print("   • Different annotation source (Last.fm tags vs crowdsourcing)")

print("\n📌 Next Steps:")
print("   1. Try more models (XGBoost, LightGBM, MLP)")
print("   2. Extract deep learning features (wav2vec2, CLAP)")
print("   3. Multimodal fusion (audio + lyrics + metadata)")
print("   4. Hyperparameter optimization and cross-validation")
print("   5. Feature importance analysis")


## 13. Save Models


In [ ]:
import joblib

# Create model save directory
MODEL_DIR = Path('../models/muse_baseline')
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Save models
models_to_save = [
    (lr_v, 'linear_regression_valence.pkl'),
    (lr_a, 'linear_regression_arousal.pkl'),
    (lr_d, 'linear_regression_dominance.pkl'),
    (ridge_v, 'ridge_valence.pkl'),
    (ridge_a, 'ridge_arousal.pkl'),
    (ridge_d, 'ridge_dominance.pkl'),
    (rf_v, 'random_forest_valence.pkl'),
    (rf_a, 'random_forest_arousal.pkl'),
    (rf_d, 'random_forest_dominance.pkl'),
    (scaler, 'feature_scaler.pkl'),
]

for model, filename in models_to_save:
    model_path = MODEL_DIR / filename
    joblib.dump(model, model_path)

# Save results
results_path = OUTPUT_DIR / 'muse_baseline_results.csv'
df_all_results.to_csv(results_path, index=False)

print("✅ All models saved!")
